In [50]:
import pandas as pd
import numpy as np
import import_ipynb
from unidecode import unidecode
import glob

import validItems

In [51]:
obras = glob.glob(r"*.xlsx")
obras

['L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx',
 'L06_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23_ARTESP_Rev00.xlsx',
 'L07_Cópia de Programação de Obras 30.01.2023 a 05.02.2023.xlsx',
 'L09_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23.xlsx']

In [52]:
def treat_list(valid_counties):
    municipalities_treated = []
    for i in range(len(valid_counties)):
        valid_counties[i] = unidecode(valid_counties[i])
        valid_counties[i] = valid_counties[i].lower()
        municipalities_treated.append(valid_counties[i])
    return municipalities_treated


def check_highway(valid_highways, worksheet_highways):
    for i in range(len(worksheet_highways)):
        if not worksheet_highways[i] in valid_highways:
            wrong_list.append(f"Rodovia errada na planilha: {obra[0:3]}| a rodovia não existe: {worksheet_highways[i]}")
            
        
def check_codes(valid_codes, worksheet_codes):
    for i in range(len(worksheet_codes)):
        if not worksheet_codes[i] in valid_codes:
            wrong_list.append(f"Cod de serviço errado na planilha: {obra[0:3]}| o codigo de serviço não existe: {worksheet_codes[i]}")


def check_counties(valid_counties, worksheet_counties):
    for i in range(len(worksheet_counties)):
        if not unidecode(worksheet_counties[i].strip()) in valid_counties:
            wrong_list.append(f"Municipio errado na planilha: {obra[0:3]}, o municipio {worksheet_counties[i].strip()}")
            

def check_coordinates(worksheet, collumn_name: str):
    comma = ","
    coordinates_list = []
    for i in range(len(worksheet)):
        coordinates_list.append(worksheet[collumn_name][i])
    comma_list = [ a for a in coordinates_list if comma in str(a)]

    if len(comma_list) != 0:
        for i in range(len(comma_list)):
            wrong_list.append(f"Coordenada errada na planilha: {obra[0:3]}| esta com , no valor: {comma_list[i]}")

def check_senses(worksheet, valid_senses):  
    for i in range(len(worksheet)):
        if not worksheet[i] in valid_senses:
            wrong_list.append(f"Sentido errado na planilha: {obra[0:3]}| o sentido não existe: {worksheet[i]}")


def check_batch(worksheet_list, valid_batch):
    for i in range(len(worksheet_list)):
        if not worksheet_list[i] in valid_batch:
            wrong_list.append(f"Lote errado na planilha: {obra[0:3]}| o lote não existe: {worksheet_list[i]}")


def check_empty(worksheet, collumn_name):
    empty_list = []
    value_empty = []
    for i in range(len(worksheet)):
        if worksheet[collumn_name][i]:
            empty_list.append(worksheet[collumn_name][i])

    for i in range(len(empty_list)):
        if "nan" in str(empty_list[i]):
            value_empty.append(str(empty_list[i]))
    if len(value_empty) > 0:
        wrong_list.append(f'temos um total de {len(value_empty)} campos vazios na {collumn_name} na planilha: {obra[0:3]}')
            
def check_km(worksheet, collumn_name: str):
    dot = "."
    comma = ","
    km_list = []
    for i in range(len(worksheet)):
        km_list.append(worksheet[collumn_name][i])
    dot_list = [a for a in km_list if dot in str(a)]
    comma_list = [ a for a in km_list if comma in str(a)]

    if len(comma_list) != 0 or len(dot_list) != 0:
        if len(comma_list) > 0:
            for i in range(len(comma_list)):
                wrong_list.append(f"na planilha: {obra[0:3]} | Erro de KM, O KM {comma_list[i]}, esta com ,")
        else:
            for i in range(len(dot_list)):
                wrong_list.append(f"na planilha: {obra[0:3]} | Erro de KM, O KM {dot_list[i]}, esta com .")
        return True
    else:
        return False

In [53]:
wrong_list = []


for obra in obras:
    df = pd.read_excel(obra, sheet_name="DADOS")
    cod_planilha = []
    rodovia_planilha = []
    municipios_planilha = []
    sentidos = []
    lote = []
    for i in range(len(df)):
        cod_planilha.append(df["CÓD. DO SERVIÇO"][i])
        rodovia_planilha.append(df["RODOVIA"][i])
        municipios_planilha.append(unidecode(df["MUNICÍPIO"][i].lower()))
        sentidos.append(df['SENTIDO'][i])
        lote.append(df["LOTE"][i])
    check_batch(lote, validItems.lotes)
    check_highway(validItems.rodovias, rodovia_planilha)
    check_codes(validItems.codigo_servico, cod_planilha)
    check_km(df, 'KM INICIAL')
    check_coordinates(df,"LAT KM INICIAL")
    check_coordinates(df,"LON KM INICIAL")
    check_km(df, 'KM FINAL')
    check_coordinates(df,"LAT KM FINAL")
    check_coordinates(df,"LON KM FINAL")
    check_senses(sentidos, validItems.sentido_validos)
    check_empty(df, "FI1")
    check_empty(df, "FI2")
    check_empty(df, "FI3")
    check_empty(df, "FI4")
    check_empty(df, "FI5")
    check_empty(df, "FI6")
    check_empty(df, "FIAC")
    check_empty(df, "ALT.")
    check_empty(df, "EMPRESA RESPONSÁVEL")
    check_empty(df, "RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA")
    check_empty(df, "ART DO RESPONSÁVEL TÉCNICO")
    check_empty(df, "NÍVEL DE SERVIÇO C/ INTERVENÇÃO")
    check_counties(treat_list(validItems.municipios), municipios_planilha)

print(wrong_list)

c:\Users\Thiago Mussolini\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Thiago Mussolini\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:296: UserWarning: Failed to load a conditional formatting rule. It will be discarded. Cause: expected <class 'float'>
  warn(msg)
c:\Users\Thiago Mussolini\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\Thiago Mussolini\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\Thiago Mussolini\AppData\Local\Programs\Python\Python310\lib\site-packages\open

['temos um total de 7 campos vazios na FI1 na planilha: L03', 'temos um total de 1 campos vazios na EMPRESA RESPONSÁVEL na planilha: L03', 'temos um total de 1 campos vazios na RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA na planilha: L03', 'temos um total de 1 campos vazios na ART DO RESPONSÁVEL TÉCNICO na planilha: L03', 'temos um total de 1 campos vazios na NÍVEL DE SERVIÇO C/ INTERVENÇÃO na planilha: L03', 'Municipio errado na planilha: L03, o municipio rio de janeiro', 'Cod de serviço errado na planilha: L07| o codigo de serviço não existe: 900', 'temos um total de 2 campos vazios na FI1 na planilha: L07', 'temos um total de 2 campos vazios na FI2 na planilha: L07']
